## 1.Loading data

In [15]:
import pandas as pd 
import numpy as np
import warnings
warnings.filterwarnings('ignore')
train_data = pd.read_csv('./train.csv')
test_data = pd.read_csv('./test.csv')
train_y = train_data["Survived"]
train_x = train_data
# Combining the training set and test
entire_data = pd.concat((train_x, test_data)).reset_index(drop=True)

## 2.Missing value fill

In [16]:
# According to the calculation of the median sale price of different positions in different ports, the port closest to the fare is filled with Embarked
Pclass1 = entire_data[entire_data['Pclass']==1][['Fare','Embarked']]
Pclass1.groupby('Embarked')['Fare'].median()
 
entire_data.loc[829,'Embarked'] = 'C'
entire_data.loc[61,'Embarked'] = 'C'


Pclass3 = entire_data[entire_data['Pclass']==3][['Fare','Embarked']]
Pclass3.groupby('Embarked')['Fare'].median()
entire_data.loc[1043,'Fare'] = 8.05


entire_data_Name = entire_data['Name']
entire_data_Family = entire_data_Name.str.split(',',expand=True)[0]
entire_data['Family'] = entire_data_Family 
just_have_Sibsp = entire_data[entire_data['Age'].isnull()==True][(entire_data['SibSp'] > 0) & (entire_data['Parch']==0)]
ticket = just_have_Sibsp['Ticket'].unique()
 
just_have_Sibsp = entire_data[entire_data['Age'].isnull()==True][(entire_data['SibSp'] > 0) & (entire_data['Parch']==0)]
ticket = just_have_Sibsp['Ticket'].unique()
 
for t in ticket:
    tempt = entire_data[entire_data['Ticket'] == t][['Ticket','Family','Age']]
    for i in tempt['Age'][entire_data['Age'].isnull() == True].index :
        index_notnull = tempt['Age'][entire_data['Age'].isnull() == False].index
        for j in index_notnull:
            if (entire_data.loc[i,'Family'] == entire_data.loc[j,'Family'] and entire_data.loc[i,'SibSp'] == entire_data.loc[j,'SibSp']):
                entire_data.loc[i,'Age'] =   entire_data.loc[j,'Age'] 
            if (entire_data.loc[i,'Parch'] == 0 and entire_data.loc[j,'Parch']== 0):
                entire_data.loc[i,'Age'] = entire_data.loc[j,'Age']
# Other age data is populated as median
entire_data['Age'] = entire_data['Age'].fillna(entire_data['Age'].median())

## 3.Adding features

In [17]:
# 1. Family Survival Rate
# 2. Age ranking by class
# 3. Ranking of fares by class
# 4. Number of relatives (simple addition)
# 5. Are there siblings
# 6. Are there children or parents
# 7. Are you alone
 
DEFAULT_SURVIVAL_VALUE = 0.5
entire_data['Family_Survival'] = DEFAULT_SURVIVAL_VALUE
 
for grp, grp_df in entire_data[['Survived','Name', 'Family', 'Fare', 'Ticket', 'PassengerId',
                           'SibSp', 'Parch', 'Age', 'Cabin']].groupby(['Family', 'Fare']):
    
    if (len(grp_df) != 1):
        for ind, row in grp_df.iterrows():
            smax = grp_df.drop(ind)['Survived'].max()
            smin = grp_df.drop(ind)['Survived'].min()
            passID = row['PassengerId']
            if (smax == 1.0):
                entire_data.loc[entire_data['PassengerId'] == passID, 'Family_Survival'] = 1
            elif (smin==0.0):
                entire_data.loc[entire_data['PassengerId'] == passID, 'Family_Survival'] = 0
                
print("Number of passengers with family survival information:", 
      entire_data.loc[entire_data['Family_Survival']!=0.5].shape[0])

for _, grp_df in entire_data.groupby('Ticket'):
    if (len(grp_df) != 1):
        for ind, row in grp_df.iterrows():
            if (row['Family_Survival'] == 0) | (row['Family_Survival']== 0.5):
                smax = grp_df.drop(ind)['Survived'].max()
                smin = grp_df.drop(ind)['Survived'].min()
                passID = row['PassengerId']
                if (smax == 1.0):
                    entire_data.loc[entire_data['PassengerId'] == passID, 'Family_Survival'] = 1
                elif (smin==0.0):
                    entire_data.loc[entire_data['PassengerId'] == passID, 'Family_Survival'] = 0
                        
print("Number of passenger with family/group survival information: " 
      +str(entire_data[entire_data['Family_Survival']!=0.5].shape[0]))

for _, grp_df in entire_data.groupby('Ticket'):
    if (len(grp_df) != 1):
        for ind, row in grp_df.iterrows():
            if (row['Family_Survival'] == 0) | (row['Family_Survival']== 0.5):
                smax = grp_df.drop(ind)['Survived'].max()
                smin = grp_df.drop(ind)['Survived'].min()
                passID = row['PassengerId']
                if (smax == 1.0):
                    entire_data.loc[entire_data['PassengerId'] == passID, 'Family_Survival'] = 1
                elif (smin==0.0):
                    entire_data.loc[entire_data['PassengerId'] == passID, 'Family_Survival'] = 0
                        
print("Number of passenger with family/group survival information: " 
      +str(entire_data[entire_data['Family_Survival']!=0.5].shape[0]))
#2
tempt  = entire_data[entire_data['Pclass'] == 1]
r = 0
pre = 0 
for i in tempt.sort_values('Age').index:
    if pre < entire_data.loc[i,'Age']: r = r + 1
    entire_data.loc[i,'Pclass1_Age_Rank'] = r
    pre = entire_data.loc[i,'Age']

tempt  = entire_data[entire_data['Pclass'] == 2]
r = 0
pre = 0
for i in tempt.sort_values('Age').index:
    if pre < entire_data.loc[i,'Age']: r = r + 1
    entire_data.loc[i,'Pclass2_Age_Rank'] = r

tempt  = entire_data[entire_data['Pclass'] == 3]
r = 0
pre = 0
for i in tempt.sort_values('Age').index:
    if pre < entire_data.loc[i,'Age']:  r = r + 1
    entire_data.loc[i,'Pclass3_Age_Rank'] = r

entire_data['Pclass1_Age_Rank'] = entire_data['Pclass1_Age_Rank'].fillna(0)
entire_data['Pclass2_Age_Rank'] = entire_data['Pclass2_Age_Rank'].fillna(0)
entire_data['Pclass3_Age_Rank'] = entire_data['Pclass3_Age_Rank'].fillna(0)
tempt = entire_data[(entire_data['Pclass'] == 1)]
r = 0
pre = 0 
for i in tempt.sort_values('Fare').index:
    if pre < entire_data.loc[i,'Fare'] : r = r + 1
    entire_data.loc[i,'Pclass1_Fare_Rank'] = r

tempt = entire_data[(entire_data['Pclass'] == 2)]
r = 0
pre = 0
for i in tempt.sort_values('Fare').index:
    if pre < entire_data.loc[i,'Fare'] : r = r + 1
    entire_data.loc[i,'Pclass2_Fare_Rank'] = r

tempt = entire_data[(entire_data['Pclass'] == 3)]
r = 0
pre = 0
for i in tempt.sort_values('Fare').index:
    if pre < entire_data.loc[i,'Fare'] : r = r + 1
    entire_data.loc[i,'Pclass3_Fare_Rank'] = r
    
entire_data['Pclass1_Fare_Rank'] = entire_data['Pclass1_Fare_Rank'].fillna(0)
entire_data['Pclass2_Fare_Rank'] = entire_data['Pclass2_Fare_Rank'].fillna(0)
entire_data['Pclass3_Fare_Rank'] = entire_data['Pclass3_Fare_Rank'].fillna(0)
entire_data['Family_Size'] = entire_data['Parch'] + entire_data['SibSp'] + 1
entire_data['Have_SibSp'] = entire_data['SibSp'].apply(lambda x:1 if x>0 else 0)
entire_data['Have_Parch'] = entire_data['Parch'].apply(lambda x:1 if x>0 else 0)
entire_data['Is_Alone'] = 1
entire_data['Is_Alone'].loc[entire_data['Family_Size'] > 1] = 0

Number of passengers with family survival information: 420
Number of passenger with family/group survival information: 546
Number of passenger with family/group survival information: 546


## 4.Remove features that do not participate in model training

In [18]:
entire_data.drop("PassengerId",axis = 1, inplace = True)
entire_data.drop("Cabin",axis = 1, inplace = True)
entire_data.drop("Ticket",axis = 1,inplace = True)
entire_data.drop("Name",axis = 1,inplace = True)
entire_data.drop("Family",axis = 1, inplace = True)
entire_data.drop("Survived",axis = 1, inplace = True)

## 5. Coding categorical variables

In [19]:
def encoder(x):
    if x == 'C':
        return 1
    elif x=='Q':
        return 2
    else:
        return 3
# Encode feature Embarked
entire_data['Embarked'] = entire_data['Embarked'].apply(lambda x: encoder(x))

entire_data['Sex'] = entire_data['Sex'].astype(str)
# One-hot encoding of category data
entire_data = pd.get_dummies(entire_data)

## 6.Divide the data into training and test sets

In [20]:
n_train = len(train_x)
train_x = entire_data[:n_train] 
test_x  = entire_data[n_train:]

## 7.Model training

In [21]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import svm,feature_selection
from sklearn.model_selection import StratifiedKFold, cross_val_score
import warnings
warnings.filterwarnings('ignore')
def acc_cv(model):
    n_folds = 5
    skf = StratifiedKFold(n_folds, shuffle=True, random_state= 8)#for shuffle data
    acc = cross_val_score(model, train_x.values,train_y, scoring="accuracy", cv = skf)
    return(acc.mean(),acc.std())
rbf_svc = make_pipeline(StandardScaler(), svm.SVC(C=1.0,random_state=2))
acc_cv(rbf_svc)

rbf_svc.fit(train_x.values,train_y.values)
test_y = rbf_svc.predict(test_x.values)

sub = pd.DataFrame()
sub['PassengerId'] = test_data['PassengerId']
sub['Survived'] = test_y
sub.to_csv('yongli_submission_0302.csv',index=False)

## 8.Report
### (1)Title link
[Titanic: Machine Learning from Disaster](https://www.kaggle.com/c/titanic)

### (2)References
[Titanic - Advanced Feature Engineering Tutorial](https://www.kaggle.com/gunesevitan/titanic-advanced-feature-engineering-tutorial/)

[Titanic Tutorial](https://www.kaggle.com/alexisbcook/titanic-tutorial)

[Titanic [0.82] - [0.83]](https://www.kaggle.com/alexisbcook/titanic-tutorial)

### (3)Grades & Ranking
Yong Li

Ranking:482

Grades:0.82296

